# **FastVLM: Efficient Vision Encoding for Vision Language Models**

https://huggingface.co/apple/FastVLM-7B

In [ ]:
# Download the model

!hf download apple/FastVLM-7B

Fetching 19 files: 100% 19/19 [00:00<00:00, 93534.95it/s]
/root/.cache/huggingface/hub/models--apple--FastVLM-7B/snapshots/e6b907da2d6ef4218cdb9f6f9d721b3e55a0a8f8


In [ ]:
# import libraries

import torch
from PIL import Image
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
MID = "apple/FastVLM-7B"
IMAGE_TOKEN_INDEX = -200  # what the model code looks for

# Load
tok = AutoTokenizer.from_pretrained(MID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MID,
    dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True,
)

# if hasattr(model, "initialize_vision_modules"):
#     model.initialize_vision_modules()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


AttributeError: 'LlavaConfig' object has no attribute 'vision_config'

In [ ]:
# Build chat -> render to string (not tokens) so we can place <image> exactly
messages = [
    {"role": "user", "content": "<image>\nDescribe this image in detail."}
]
rendered = tok.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=False
)
pre, post = rendered.split("<image>", 1)

# Tokenize the text *around* the image token (no extra specials!)
pre_ids  = tok(pre,  return_tensors="pt", add_special_tokens=False).input_ids
post_ids = tok(post, return_tensors="pt", add_special_tokens=False).input_ids

# Splice in the IMAGE token id (-200) at the placeholder position
img_tok = torch.tensor([[IMAGE_TOKEN_INDEX]], dtype=pre_ids.dtype)
input_ids = torch.cat([pre_ids, img_tok, post_ids], dim=1).to(model.device)
attention_mask = torch.ones_like(input_ids, device=model.device)

# Preprocess image via the model's own processor
img = Image.open("/content/label_0_1000.jpeg").convert("RGB")
px = model.get_vision_tower().image_processor(images=img, return_tensors="pt")["pixel_values"]
px = px.to(model.device, dtype=model.dtype)

In [ ]:
# Generate
with torch.no_grad():
    out = model.generate(
        inputs=input_ids,
        attention_mask=attention_mask,
        # Ensure image pixel values are on the correct device and dtype
        images=px.to(model.device, dtype=model.dtype),
        max_new_tokens=128,
    )
print(tok.decode(out[0], skip_special_tokens=True))

NotImplementedError: Cannot copy out of meta tensor; no data!